In [1]:
%%capture
!pip install pypdf2 tabula-py

In [2]:
import pandas as pd
import re
import tabula as tb

from google.colab import drive
from math import isnan
from PyPDF2 import PdfReader

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def get_title_cleanup() -> dict:
  return {
    'Senior Advisors': 'Senior Advisor',
    'Technology Specialists': 'Technology Specialist',
    'Special Assistants to the General Manager': 'Special Assistant to the General Manager',
    'Player Development Coordinators': 'Player Development Coordinator',
    'Minor League Video Coordinators': 'Minor League Video Coordinator',
    'Corporate Partnerships Sales Directors': 'Corporate Partnerships Sales Director',
    'Corporate Partnerships Sales Managers': 'Corporate Partnerships Sales Manager',
    'Suite Sales Account Executives': 'Suite Sales Account Executive',
    'Promotions & Special Events Coordinators': 'Promotions & Special Events Coordinator',
    'Media Relations Coordinators': 'Media Relations Coordinator',
    'Security Managers': 'Security Manager',
    'Park Operations Coordinators': 'Park Operations Coordinator',
    'Park Operations Assistants': 'Park Operations Assistant',
    'Ballpark Electricians': 'Ballpark Electrician',
    'Housekeeping Supervisors': 'Housekeeping Supervisor',
    'Producer Editors': 'Producer Editor',
    'New Business Representatives': 'New Business Representative',
    'Group Sales Senior Executives': 'Group Sales Senior Executive',
    'Group Sales Executives': 'Group Sales Executive',
    'Client Services Representatives': 'Client Services Representative',
    'Ticket Services Managers': 'Ticket Services Manager',
    'Box Office Administrators': 'Box Office Administrator',
    'Sales Representatives': 'Sales Representative',
    'Account Executives, Client Services': 'Account Executive, Client Services',
    'Account Executives, Group Sales': 'Account Executive, Group Sales',
    'Account Executives, New Business': 'Account Executive, New Business',
    'Accout Executives, Premium Sales': 'Account Executive, Premium Sales',
    'Analysts, Baseball Operations': 'Analyst, Baseball Operations',
    'Assistant Athletic Trainers': 'Assistant Athletic Trainer',
    'Coordinators, Corporate Partnership Activation': 'Coordinator, Corporate Partnership Activation',
    'Corporate Recruiters': 'Corporate Recruiter',
    'Full-Stack Software Engineers': 'Full-Stack Software Engineer',
    'Managers, Corporate Partnership Activation': 'Manager, Corporate Partnership Activation',
    'Managers, Ticket Services': 'Manager, Corporate Ticket Services',
    'Operations Managers': 'Operations Manager',
    'Promotions & Special Events Coordinators': 'Promotions & Special Events Coordinator',
    'Sales Executives, Premium Sales': 'Sales Executive, Premium Sales',
    'Sales Managers, Corporate Partnerships': 'Sales Manager, Corporate Partnerships',
    'Security Supervisors': 'Security Supervisor',
    'Senior Account Executives, Client Services': 'Senior Account Executive, Client Services',
    'Senior Analysts, Baseball Operations': 'Senior Analyst, Baseball Operations',
    'Solutions Managers, Corporate Partnerships': 'Solutions Manager, Corporate Partnerships',
    'Community Impact Managers': 'Community Impact Manager',
    'HVAC Technicians': 'HVAC Technician',
    'HVAC Techs': 'HVAC Technician',
    'Assistant Athletic TrainerCLUB DIRECTORY (continued)': 'Athletic Trainer'
  }

In [4]:
def get_department_cleanup() -> dict:
  return {
    # "bad department name": "good department name"
    'Media Guide 2019 25': None,
    'PERSONNELFRONT OFFICE DIRECTORY': None,
    '26 Cleveland Indians CLUB DIRECTORY (continued)': None,
    'Media Guide 2019 27': None,
    'PERSONNELCORPORATE PARTNERSHIPS & PREMIUM HOSPITALITY': 'CORPORATE PARTNERSHIPS & PREMIUM HOSPITALITY',
    '28 Cleveland Indians LIVE EXPERIENCE': 'LIVE EXPERIENCE',
    'Media Guide 2019 29': None
  }

In [5]:
def get_start_index(year:int, team:str, page:int) -> int:
  if year == 2019 and team == 'DET' and page == 6:
    return 8
  else:
    return 0

In [6]:
def cleanup_name(original_name:str) -> str:

  new_name = original_name.replace('PERSONNEL', '')

  # Remove banned substrings
  banned_substrings = [
      'Dr.', 'O.D.', 'OD', 'M.D.', 'MD', 'D.O.', 'LMT', 'PA-C', 'D.D.S', 'DDS', 
      'D.C.', 'DC', 'FCOVD'
  ]

  for e in banned_substrings:
    new_name = new_name.replace(e, '')


  # Trim unwanted spaces and characters
  new_name = ''.join(new_name.split(',')) \
               .strip(' ') \
               .strip('.') \
               .strip(' ') \
               .replace("' ", "'")

  return new_name

In [7]:
def get_full_team_name(year:int) -> str:
  if year >= 2022:
    return 'Cleveland Guardians'
  else:
    return 'Cleveland Indians'

In [14]:
def get_front_office_roster(pdf:PdfReader, year:int, team:str, first_page:int, last_page:int) -> pd.DataFrame:

  def get_department(row):
    try:
      if isnan(row['Title']):
        return row['Name']
      else:
        return None

    except:
      return None

  df_main = pd.DataFrame()
  pages = range(first_page + 1, last_page + 2)

  for p in pages:
    pdf_page = pdf.getPage(p)
    start = get_start_index(year, team, p)

    records = [record for record in pdf_page.extractText().split('\n\n')]
    

    if year == 2020:
      records = [record for record in pdf_page.extractText().split('\n')]
      title_name_split = [re.split(' ?\.\.+ ?', record) for record in records]
      df = pd.DataFrame(records).loc[7:]
      # df = pd.DataFrame({
      #   'name_title': df.loc[7:, 1].astype(str) + df.loc[7:, 0].shift(-1).astype(str)
      # })

      # df = df[df.name_title != 'None']
      # df['name_title'] = df['name_title'].str.replace('None', '')

      # df['name_title'] = df['name_title'].apply(lambda x: cleanup_name(x))
      # df['name_title'] = df['name_title'].str.rsplit(' ', 2)
    else:
      title_name_split = [re.split(' ?\.\.+ ?', record) for record in records]

    # df = pd.DataFrame(title_name_split)

    # # # Cleanup job titles
    # df['Title'] = df['Title'].str.strip()
    # df['Title'] = df['Title'].replace(get_title_cleanup())
    # df['Prior Title'] = df.Title.shift(1)


    # # # Cleanup names
    # df['Name'] = df.Name.apply(lambda x: cleanup_name(x))
    

    # # Record the source of the information
    # df['Source of Information'] = f'Cleveland Indians, pp {first_page}-{last_page}'


    # # # Expand name tuples to multiple rows, if necessary
    # # df = df.explode('Name', ignore_index=True)


    # # # Determine the department
    # df['Department'] = df.apply(lambda x: get_department(x), axis=1) \
    #                      .replace(get_department_cleanup()) \
    #                      .str.title()
    # df['Department'] = df['Department'].ffill()


    # # # Remove empty rows
    # df = df[~df.Team.isna()]
    

    # # # Order the columns as specified by SABR BoB Committee,
    # df = df[['Year', 'Team', 'Name', 'Title', 'Department', 'Source of Information']]


    # # Update the main dataframe
    df_main = pd.concat([df_main, df]).reset_index(drop=True)

  return df_main

In [15]:
pages = {
  'CLE': {
    # 2019: [27,31]
    2020: [26,30]
    # 2021: [41,45]
  }
}

final_df = pd.DataFrame()

for team, contents in pages.items():
  for year, pdf_pages in contents.items():
    first_page = pdf_pages[0]
    last_page = pdf_pages[1]
    temp_df = get_front_office_roster(
                PdfReader(f'drive/My Drive/sabr/business-of-baseball/team-media-guides/{team}/{year}_{team}_Media_Guide.pdf'),
                year, team, first_page, last_page
              )
    
    final_df = pd.concat([final_df, temp_df])
    # final_df['Department'].ffill(inplace=True)

In [16]:
final_df

,0
0,................................................
1,ant General Manager Bob DiBiasio
2,................................................
3,"Vice President, Public Affairs Alex King"
4,................................................
...,...
429,g Physician Jamie Starkey
430,................................................
431,"rative Medicine Don Franklin, D.C."
432,................................................


In [526]:
isnan(final_df.loc[59])

True

In [428]:
c = final_df.columns
f = lambda x: pd.Series(x.dropna().to_numpy()).reindex(range(len(c)))
final_df[c] = final_df[c].apply(f, axis=1)

In [429]:
final_df

,0,1,2
0,Media Guide 2020 25,NaN,NaN
1,PERSONNELFRONT OFFICE DIRECTORY,NaN,NaN
2,EXECUTIVE OFFICE,NaN,NaN
3,Paul J. Dolan,Owner/Ch airman/Chief Executive Officer Chris ...,NaN
4,,"Presiden t, Baseball Operations Brian Barren","President, Business Operations Mike Chernoff"
...,...,...,...
464,g Physician Jamie Starkey,NaN,NaN
465,,Integ,NaN
466,"rative Medicine Don Franklin, D.C.",NaN,NaN
467,,Chiro,NaN


In [462]:
df['title'] = df['name_title'].apply(lambda x: x[0]).shift(-2)
df['name'] = df['name_title'].apply(lambda x: ' '.join(x[1:]))

In [463]:
df

,name_title,title,name
7,"[Assistant General Manager, Bob, DiBiasio]","Senior Vice President, Public Affairs",Bob DiBiasio
8,[nan],nan,
9,"[Senior Vice President, Public Affairs, Alex, ...","Senior Vice President, Marketing & Strategy",Alex King
10,[nan],nan,
11,"[Senior Vice President, Marketing & Strategy, ...","Senior Vice President, Technology & Chief Info...",Neil Weiss
...,...,...,...
464,[nan],nan,
465,"[Integrative Medicine Don, Franklin,, D.C.]",Chiropractor,"Franklin, D.C."
466,[nan],nannan,
467,[Chiropractor],NaN,
